# Bayesian Parameter Estimation of a Fluorescence-to-Protein Calibration Factor 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import emcee
import pandas as pd
import theano.tensor as tt
import mscl_utils as mscl
import mscl_mcmc as msmc
import multiprocessing 
import bokeh.io
import corner
import scipy.special
import scipy.optimize
import scipy.stats
import bokeh.plotting
from scipy.signal import gaussian, convolve
bokeh.io.output_notebook()
mscl.set_plotting_style()
%matplotlib inline
num_cpus = multiprocessing.cpu_count()

Loading BokehJS ...

## Counting By Measuring Fluctuations 

Imagine that we have a cell that has a fixed number of fluorescent proteins.  As this cell divides, these proteins will be Binomially partitioned into the two daughter cells. By examining the difference in fluorescence between the two daughters, we can determine just how "bright" a single molecule should be. 

We can begin by positing that the fluorescence is conserved (production and degradation rates are 0). Mathematically, we can say that 

$$
I_\text{tot} = I_1 + I_2 \tag{1}
$$

where $I_\text{tot}$ is the total fluorescence of the mother cell and $I_1$ and $I_2$ are the intensities of the two daugher cells. As we know this fluorescence comes from a mix of the background fluorescence and the fluorescence of the proteins themselves, we can write that the total intensitiy of the mother cell should be proportional to the total number of proteins $N_\text{tot}$,

$$
I_\text{tot} = \alpha N_\text{tot}, \tag{2}
$$

assuming (for now) that the background fluorescence and the error in our measurement is zero. Assuming that there is a single fluroescence calibration factor and that it has the same value from cell to cell, we can write Eq. 2 for the daughter cells as

$$
I_1 = \alpha N_1\,\, ; \,\, I_2 = \alpha N_2  \tag{3, 4}
$$

which follows the assumption that the protein copy number is conserved as well,

$$
N_\text{tot} = N_1 + N_2 \tag{5}.
$$

When a cell divides, the distribution of the proteins into the two daughter cells is binomial with a partitioning probability $p$,

$$
P(n \, \vert \, N_\text{tot}) = {N_\text{tot}! \over n! (N_\text{tot} - n)!} p^{n}(1 - p)^{N_\text{tot} - n}, \tag{6}
$$

Where we've generalized $N_1$ or $N_2$ as $n$. To examine how the proteins were partitioned amongst the daughter cells, we can look at how different the intensities are between them and relate it to the total intensity of the mother as 

$$
\langle(I_1 - I_2)^2\rangle = \langle (2I_1 - I_\text{tot})^2 \tag{7}
$$

which we can translate to protein copy numbers as 

$$
\langle(I_1 - I_2)^2\rangle = \langle\left(2\alpha N_1 - \alpha N_\text{tot}\right)^2\rangle. \tag{8}
$$

As we are discussing averages in this context, we can examine this more generally as

$$
\langle(I_1 - I_2)^2 \rangle = \alpha(2\langle n \rangle - N_\text{tot})^2 \tag{9}
$$

To finish our calculation, we will have to know the mean and variance of the protein copy number upon division. 





While these two moments of the Binomial distribution are well know, it's useful to derive them explicitly. The mean can be calculated as

$$
\langle n \rangle = \sum\limits_{n = 0}^{N_\text{tot}}n{{N_\text{tot}}\choose{n}}p^{n}(1-p)^{N_\text{tot} - n}. \tag{10}
$$

This can be simplified by realizing that

$$
n {N_\text{tot}! \over n!(N_\text{tot} - n)!} = {N_\text{tot}! \over (n-1)!(N_\text{tot} - n)!} = {N_\text{tot}(N_\text{tot} - 1)! \over (n - 1)!(N_\text{tot} - n)!} \tag{11}
$$

We can now rewrite Eq. 10 as

$$
\langle n \rangle = N_\text{tot} p \sum\limits_{n= 1}^{N_\text{tot}}{{N_\text{tot} - 1}\choose{n - 1}}p^{n - 1}(1 - p)^{N_\text{tot} - n} \tag{12}
$$

By defining $\nu = N_\text{tot} - 1$ and $k = n - 1$, we yield. 

$$
\langle n \rangle = N_\text{tot} p \overbrace{\sum\limits_{k= 0}^{\nu}{{\nu}\choose{k}}p^k(1 - p)^{\nu - k}}^\text{1} = N_\text{tot} p \tag{13}
$$

which is what we would naïvely expect. With this in hand, we can naïvely solve for the variance. We can solve for $\langle n^2 \rangle$ as we did above by realizing 

$$
\langle n^2 \rangle = N_\text{tot}p\sum\limits_{n=1}^{N_\text{tot}}n{{N_\text{tot} - 1}\choose{n -1}}p^{n-1}(1 - p)^{N_\text{tot} - n} = N_\text{tot}p\sum\limits_{k=0}^\nu(k + 1){{\nu}\choose{k}}p^k(1-p)^{\nu - k}  \tag{14},
$$

where we pull the same trick of reparameterizing Eq. 13 for $\nu = N_\text{tot} - 1$ and $k = n - 1$. Simplifying Eq. 14 generates 

$$
\langle n^2 \rangle = (N_\text{tot}p)^2 + N_\text{tot}p(1 - p) \tag{15}
$$

allowing us to express the variance $\sigma^2$ as 

$$
\sigma^2 = \langle n^2 \rangle - \langle n \rangle^2 = N_\text{tot}p(1 - p). \tag{16}
$$



We can make our calculation a bit less verbose aby assuming that partitioning of the proteins is always fair such that $p = 0.5$. This means our mean and variance can be more simply written as 

$$
\langle n \rangle = {N_\text{tot} \over 2}\, ; \, \langle n^2 \rangle= {N_\text{tot}+ N_\text{tot}^2 \over 4}. \tag{17, 18}
$$

We can now return to Eq. 9 and include the mean copy number,

$$
\begin{align}
\langle (I_1 - I_2)^2 \rangle &= 4\alpha^2(\langle n^2 \rangle - \langle n \rangle N_\text{tot}) + (\alpha N_\text{tot})^2\\
& = 4\alpha^2\left({N_\text{tot} + N_\text{tot}^2 \over 4} - {2N_\text{tot}^2 \over 4}\right) + \alpha^2N_\text{tot}^2 \tag{19}.
\end{align}
$$

Some simplification brings us to our result, 

$$
\langle (I_1 - I_2)^2 \rangle = \alpha^2 N_\text{tot} = \alpha I_\text{tot}. \tag{20}
$$
 
This result tells us that the squared difference in the intensity between any two daughter cells should be linearly related to the intensity of the mother cell with a slope of the calibration factor $\alpha$. While this is a very beautiful result, our measurement is a bit more difficult.

## Incorporating Measurement Error

While we were rewarded for our mathematical efforts with Eq. 20, the practical use of this theoretical prediction is a bit more complicated. Primarily, we must be able to incoporate our error in the measurement of the single cell intensities. This means that our Eq. 20 becomes. 

$$
I_\text{tot} = \alpha_0 + \alpha N_\text{tot} + \epsilon \tag{21}
$$

where $\alpha_0$ is the background fluorescence of our sample and $\epsilon$ is our measurement error. In [Rosenfeld et al. 2005]() and [Brewster et al. 2014](), the calibration factor was determined by binning the observed data and then fitting the prediction to the mean, assuming the measurement error was 0. While this is an acceptable approach, choosing the appropriate bin width is difficult and can bias the results. A more robust approach would be to write a heirarchical model which models each cell division event individually and estimate a hyperparameter for the calibration factor. 



### Taking a Bayesian approach

We are interested in estimating two calibration parameters - $\alpha_0$ and $\alpha$ - and (as a consequence), the most likely protein copy numbers $N_1$ and $N_\text{tot}$. As is usually the case, we can begin by writing down Bayes' theorem for the problem, 

$$
P(\alpha_0, \alpha, \sigma, N_1, N_\text{tot}\,\vert\, I_1, I_2) = {P(I_1, I_2\, \vert \, \alpha_0, \alpha, \sigma, N_1, N_\text{tot})P(\alpha_0, \alpha, \sigma, N_1, N_\text{tot}) \over P(I_1, I_2) }. \tag{22}
$$

in which we've introduced the measurement error as $\sigma$ as it's related to the normal distribution of the measurements. Our likelihood can be broken up into three parts $--$ one for $I_1$, for $I_2$, and one for $[N_1, N_\text{tot}]$. As we've discussed already, the partitioning of the proteins into the daughter cells is binomial, allowing us to write

$$
P(N_1\, \vert\, N_\text{tot}, p) = {{N_\text{tot}}\choose{N_1}}p^{N_1}(1 - p)^{N_\text{tot} - N_1}. \tag{23}
$$

By assuming that partitioning is *always* fair (which may be a big assumption), we can simplify Eq. 23 to be

$$
P(N_1\, \vert \, N_\text{tot}, p) = {{N_\text{tot}}\choose{N_1}}2^{-N_\text{tot}}. \tag{24}
$$

It is likely that the measured intensity of a given cell is normally distributed. We can then say that for $I_1$, the  likelihood is gaussian with a variance $\sigma$,

$$
P(I_1 \,\vert\, \alpha_0, \alpha, N_1) = {1 \over \sqrt{2\pi \sigma^2}}\exp\left[{-(I_1 - (\alpha_0 + \alpha N_1)^2 \over 2\sigma^2}\right]. \tag{25}
$$

Similarly, we can write the likelihood for $I_2$ as 

$$
P(I_2 \,\vert\, \alpha_0, \alpha, N_1, N_\text{tot}) = {1 \over \sqrt{2 \pi \sigma^2}}\exp\left[{-(I_2 - (\alpha_0 + \alpha (N_\text{tot} - N_1))^2 \over 2 \sigma^2}\right] \tag{26}
$$

where we've again used the fact that protein copy number is conserved $N_\text{tot} = N_1 + N_2$. 

The priors can be a little more tricky to figure out. To begin simply, we can assume a uniform uninformative prior for the protein copy numbers $N_1$ and $N_\text{tot}$ using our knowledge that they must be discrete,

$$
P(N_1, N_\text{tot}) = \begin{cases} {1 \over n_\text{max} - n_\text{min}} && n \in \mathbb{Z}\, ;\, n_\text{min} \leq n \leq n_\text{max} \,;\, N_1 \leq N_\text{tot}\\
0 && \text{otherwise}
\end{cases}. \tag{27}
$$

Similarly, we can choose uniform priors for our two calibration parameters $\alpha_0$ and $\alpha$ as 

$$
P(\alpha_0, \alpha) = \begin{cases} {1 \over \alpha_\text{max} - \alpha_\text{min}} && \alpha_\text{min} \leq \alpha \leq \alpha_\text{max}\\
0 && \text{otherwise} 
\end{cases}. \tag{28}
$$

As is typically the case, we can assign an improper (i.e. unnormalized) Jeffrey's prior for $\sigma$,

$$
P(\sigma) = {1 \over \sigma}. \tag{29}
$$

With likelihoods and priors, we are ready to write down the complete posterior. For ease, we can ignore the evidence $P(I_1, I_2)$. As we are looking over $k$ cell divisions, we have a set of $[N_1, N_\text{tot}]$ and $[I_1, I_2]$,  

$$
P(\alpha_0, \alpha, \sigma, [N_1, N_\text{tot}]\,\vert\,I_1, I_2) \propto P(\alpha, \alpha_0, [N_1, N_\text{tot}]){1 \over \sigma}\prod\limits_{i}^{k}{{N_{\text{tot},i}}\choose{N_{1,i}}}2^{-N_{\text{tot},i}}\times
\left({1 \over 2\pi \sigma^2}\right)^k\exp\left[{1 \over 2\sigma^2}\sum\limits_{i}^{k}\left((I_{1,i} - (\alpha_0 + \alpha N_{1,i}))^2 + (I_{2, i} - (\alpha_0 + \alpha(N_{\text{tot},i} - N_{1,i})))^2\right)\right], \tag{30}
$$

in which the priors for $\alpha_0, \alpha,$ and $[N_1, N_\text{tot}]$ were left out for notational clarity. 

While we could continue on to sample this distribution using MCMC, we have to be very careful of the priors. For example, with a $2k + 3$ parameter sampling, any degeree of information included in our prior will be greatly amplified and can steer the sampling off into other low-probabability local minima. We can reduce this issue by marginalizing Eq. 30 over $\sigma$,

$$
P(\alpha_0, \alpha, [N_1, N_\text{tot}] \, \vert\, [I_1, I_2]) = \int\limits_0^\infty d\sigma P(\alpha_0, \alpha, \sigma, [N_1, N_\text{tot}]\,\vert\,[I_1, I_2]). \tag{31}
$$

While this is a gnarly integral, the result is well studied and is the Student-t distribution, 

$$
\begin{align}
P(\alpha_0, \alpha, [N_1, N_\text{tot}] \, \vert \, [I_1, I_2]) \propto &\prod\limits_i^k {{N_{\text{tot},i}}\choose{N_{1,i}}}2^{-N_{\text{tot},i}}\times\\
&\left(\sum\limits_i^k \left((I_{1,i} - \alpha_0 - \alpha N_{1,i}))^2 + (I_{2,i} - \alpha_0 - \alpha(N_{\text{tot},i} - N_{1,i}))^2\right)\right)^{-k / 2}. \tag{32}
\end{align}
$$

Rather than jumping immediately into analyzing *real* data, we can test it on something we know.

### Analyzing simulated data

To ensure that this type of inference is appropriate, we will generate a fake data set in which the calibration factor and the background fluorescence is known *a priori*. We can choose values that are physically realistic, as is done in the code cell below.

In [18]:
# Set the parameters to generate the data set.
n_cells = 500
max_proteins = 100 
alpha_opt = 200 
alpha_sigma = 5
alpha_0_opt = 10
alpha_0 = np.random.normal(10, 2, size=n_cells)
alpha_seed = np.random.normal(alpha_opt, alpha_sigma, size=n_cells)

# Set the parent protein copy numbers and partition into children.
n_tot = (np.random.rand(n_cells) * max_proteins).astype(int)
n_1 = np.random.binomial(n_tot, p=0.5)
n_2 = n_tot - n_1

# Compute the intensities.
I_1 = alpha_seed * n_1 
I_2 = alpha_seed * n_2
I_tot = alpha_seed * n_tot

# Make it a DataFrame and compute the quantity of interest.
mothers = np.arange(0, n_cells, 1)
df = pd.DataFrame(np.array([n_tot, n_1, n_2, I_1, I_2, I_tot, mothers]).T, columns=['n_tot', 'n_1', 'n_2',
                                                         'I_1', 'I_2', 'I_tot', 'cell_no'])
df.loc[:, 'sq_diff'] = (df['I_1'] - df['I_2'])**2

# Generate the prediction from the theory.
I_tot_range = np.logspace(2, 5, 300)
prediction = alpha_opt * I_tot_range

To prove that this synthetic data is in line with our derivation above, we can plot the raw points along with Eq. 20. 

In [19]:
# Instantiate the figure and add axis labels. 
p = mscl.bokeh_boiler(plot_width=600, plot_height=400, x_axis_type='log',
                      y_axis_type='log', x_axis_label='I\u2081 + I\u2082',
                      y_axis_label='(I\u2081 - I\u2082)\u00B2')

# Plot the data and the theoretical prediction.
p.circle(x=df['I_tot'], y=df['sq_diff'],color='slategray', 
         alpha=0.4, legend='simulated data')
p.line(x=I_tot_range, y=prediction, color='tomato',
      legend='α\u2080 + α(Ι\u2081 + I\u2082)', line_width=2)

# Add a legend and show the plot
p.legend.location='top_left'

bokeh.io.show(p)

As expected, our theoretical result matches the simulated data. To sample Eq. 32, we will use the package [PyMC3]() which has a wide variety of samplers and simple tools for constructing complicated models.

###  A Heirarchical model

In [ ]:
with pm.Model() as model:
    # Set up the hyper prior for alpha.
    alpha_mu = pm.Uniform('alpha_mu', lower=0, upper=1000)
    alpha_sig = msmc.Jeffreys('alpha_sig', lower=1E-3, upper=1E6) 
    
    # Define the other priors.
    alpha = pm.

### Figuring out where to start 

In [20]:
# Bin the data by a given event size. 
bin_size = 80
# Sort the x data. 
sorted_I_tot = np.sort(df['I_tot'])

# Set up binning range
bins = np.arange(0, len(sorted_I_tot) + bin_size, bin_size)

# Compute the means of each. 
mean_sum, mean_sqdiff = [], []
df.sort_values(by='I_tot', inplace=True)
for i in range(1, len(bins)):
    slc_data = df.iloc[bins[i-1]: bins[i]]
    mean_sum.append(slc_data['I_tot'].mean())
    mean_sqdiff.append(slc_data['sq_diff'].mean())

In [21]:
def bin_and_fit(df, x_var='I_tot', y_var='sq_diff', bin_size=50):
    # Sort the data
    df_sorted = df.sort_values(by=x_var)
    
    # Compute the bins. 
    bins = np.arange(0, len(df_sorted) + bin_size, bin_size)
    # Set up storage vectors.
    mean_x, mean_y = [], []
    


In [22]:
def theo(I_tot_range, alpha):
    return alpha * I_tot_range
popt, junk = scipy.optimize.curve_fit(theo, mean_sum, mean_sqdiff, p0=1000)
slope = popt[0]

With binning, now we can perform a regression t figure out the best starting value. 

In [23]:
# Instantiate the figure and add axis labels. 
p = mscl.bokeh_boiler(plot_width=600, plot_height=400, x_axis_type='log',
                      y_axis_type='log', x_axis_label='I\u2081 + I\u2082',
                      y_axis_label='(I\u2081 - I\u2082)\u00B2')

# Plot the data and the theoretical prediction.
p.circle(x=df['I_tot'], y=df['sq_diff'],color='slategray', 
         alpha=0.4, legend='simulated data')
p.line(x=I_tot_range, y=prediction, color='tomato',
      legend='α\u2080 + α(Ι\u2081 + I\u2082)', line_width=2)
p.circle(x=mean_sum, y=mean_sqdiff, color='dodgerblue', legend='binned points')
p.line(x = I_tot_range, y=slope * I_tot_range, line_width=2, 
       color='dodgerblue', legend='line from linear regression')

# Add a legend and show the plot
p.legend.location='top_left'

bokeh.io.show(p)

### Implementation via the MCMC hammer 

In [25]:
def log_marginalized_normal(params, i1, i2):
    """
    Computes an (unnormalized) Student-t distribution as the result of 
    marginalization over σ from the normal distribution for.
    
    Parameter
    ---------
    params : tuple of nd-arrays.
        Tuple or list of the parameters. They should be given in the 
        following order.
        
    """ 
    # All parameters must be positive
    if (params < 0).any():
        return -np.inf

    # Extract parameters
    n = len(i1)
    alpha = params[0] 
    N1 = params[1:n+1]
    Ntot = params[n+1:]
    
    # Make sure N1 and Ntot are physical
    if (N1 > Ntot).any():
        return -np.inf
    
    # Return log posterior
    return -Ntot.sum() * np.log(2) \
            + scipy.special.gammaln(Ntot+1).sum() \
            - scipy.special.gammaln(N1+1).sum() \
            - scipy.special.gammaln(Ntot-N1+1).sum() \
            - n / 2 * np.log(((i1 - (alpha_0 + alpha * N1))**2 
                              + (i2 - (alpha_0 + alpha * (Ntot - N1)))**2).sum())



In [26]:
# Set the starting positions given our optimal value of alpha.
N1_0 = df['I_1'].values / alpha_opt
Ntot_0 = df['I_tot'].values / alpha_opt

# Set up MCMC problem
n = len(df['I_1'])
n_dim = 2 * len(df['I_1']) + 1
n_walkers =  3 * n_dim + 1
n_burn = 5000 
n_steps = 5000 
thin = 50  # Only store every 100 samples

# p0[i,j] is the starting point for walk i along variable j.
# Ensure we start with Ntot >= N1
p0 = np.empty((n_walkers, n_dim))
p0[:,0] = np.random.normal(slope, 1, n_walkers)
# alpha_opt + \
#         np.random.uniform(-alpha_opt*0.01, alpha_opt*0.01, n_walkers)
p0[:,1:n+1] = N1_0 + \
        np.random.uniform(0, N1_0*0.01, size=(n_walkers, n))
p0[:,n+1:] = Ntot_0 + \
        np.random.uniform(Ntot_0*0.01, Ntot_0*0.02, size=(n_walkers, n))

# Instantiate sampler 
sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_marginalized_normal, 
                                args=(df['I_1'], df['I_2']), threads=46)

In [27]:
print('Doing burn-in', flush=True)
pos, prob, state = sampler.run_mcmc(p0, n_burn, storechain=False)

# Sample again, starting from end burn-in state
print('sampling', flush=True)
_ = sampler.run_mcmc(pos, n_steps, thin=thin)
print('finished!')

Doing burn-in
sampling
finished!


In [28]:
_ = corner.corner(sampler.flatchain[:, 0:2])

ValueError: It looks like the parameter(s) in column(s) 1 have no dynamic range. Please provide a `range` argument.

In [ ]:
def gelman_rubin(chain):
    ssq = np.var(chain, axis=1, ddof=1)
    W = np.mean(ssq, axis=0)
    θb = np.mean(chain, axis=1)
    θbb = np.mean(θb, axis=0)
    m = chain.shape[0]
    n = chain.shape[1]
    B = n / (m - 1) * np.sum((θbb - θb)**2, axis=0)
    var_θ = (n - 1) / n * W + 1 / n * B
    r_hat = np.sqrt(var_θ / W)
    return r_hat

In [ ]:
chains = [c for c in sampler.chain[:, :, :]]
for c in chains:
    a = gelman_rubin(c)
    print(a)

In [ ]:
np.shape(sampler.chain)

In [ ]:
sampler.chain[1]

In [ ]:
p1 = mscl.bokeh_boiler(width=400)
p2 = mscl.bokeh_boiler(width=400)
x, y = mscl.ecdf(sampler.flatchain[:, 0])
p1.line(x, y, color='dodgerblue')

hist, bins = np.histogram(sampler.flatchain[:, 0], bins=1000)
p2.quad(bottom=0, top=hist, left=bins[1:], right=bins[:-1], color='dodgerblue', 
      alpha=0.5)

grid = bokeh.layouts.gridplot([[p1, p2]])
bokeh.io.show(grid)